In [212]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

ImportError: cannot import name 'AnnotationBbox' from 'matplotlib' (/Users/dpelleg/opt/anaconda3/lib/python3.8/site-packages/matplotlib/__init__.py)

In [287]:
img = cv.imread('noisy rect.png',0)
img_orig = img.copy()
template_orig = cv.imread('lech.png',0)
template_orig = cv.imread('lech-simple.png',0)
template_orig = cv.imread('line.png',0)
template_orig = cv.imread('vertical.png',0)

In [ ]:
w, h = template_orig.shape[::-1]
scale_percent = 60 # percent of original size
width = int(w * scale_percent / 100)
height = int(h * scale_percent / 100)
dim = (width, height)
template = cv.resize(template_orig, dim, interpolation = cv.INTER_AREA) 
# All the 6 methods for comparison in a list
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
for meth in methods:
    img = img2.copy()
    method = eval(meth)
    # Apply template Matching
    res = cv.matchTemplate(img,template,method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv.rectangle(img,top_left, bottom_right, 255, 2)
    plt.subplot(121),plt.imshow(res,cmap = 'gray')
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(img,cmap = 'gray')
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    plt.suptitle(meth)
    plt.show()

In [60]:
from PIL import Image, ImageDraw
import gpxpy as mod_gpxpy
from PIL import Image as mod_pil_image
from PIL import ImageDraw as mod_pil_draw
import glob as mod_glob
import math as mod_math
import numpy
osm_tile_res = 256

In [29]:
def format_time(time_s):
    if not time_s:
        return 'n/a'
    minutes = mod_math.floor(time_s / 60.)
    hours = mod_math.floor(minutes / 60.)
    return '%s:%s:%s' % (str(int(hours)).zfill(2), str(int(minutes % 60)).zfill(2), str(int(time_s % 60)).zfill(2)) 

In [57]:
def get_map_suffix ():
#    return "osm-cycle"
#    return "seznam-turist"
    return "roads"


In [38]:
def osm_lat_lon_to_x_y_tile (lat_deg, lon_deg, zoom):
    """ Gets tile containing given coordinate at given zoom level """
    # taken from http://wiki.openstreetmap.org/wiki/Slippy_map_tilenames, works for OSM maps and mapy.cz
    lat_rad = mod_math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - mod_math.log(mod_math.tan(lat_rad) + (1 / mod_math.cos(lat_rad))) / mod_math.pi) / 2.0 * n)
    return (xtile, ytile)


def osm_get_auto_zoom_level ( min_lat, max_lat, min_lon, max_lon, max_n_tiles):
    """ Gets zoom level which contains at maximum `max_n_tiles` """
    for z in range (0,17):
        x1, y1 = osm_lat_lon_to_x_y_tile (min_lat, min_lon, z)
        x2, y2 = osm_lat_lon_to_x_y_tile (max_lat, max_lon, z)
        max_tiles = max (abs(x2 - x1), abs(y2 - y1))
        if (max_tiles > max_n_tiles):
            print ("Max tiles: %d" % max_tiles)
            return z 
    return 17


In [55]:
class MapCreator:
    """ Class for map drawing """

    def __init__(self, min_lat, max_lat, min_lon, max_lon, z):
        """ constructor """
        x1, y1 = osm_lat_lon_to_x_y_tile (min_lat, min_lon, z)
        x2, y2 = osm_lat_lon_to_x_y_tile (max_lat, max_lon, z)
        self.x1 = min (x1, x2)
        self.x2 = max (x1, x2)
        self.y1 = min (y1, y2)
        self.y2 = max (y1, y2)
        self.w = (self.x2 - self.x1 + 1) * osm_tile_res
        self.h = (self.y2 - self.y1 + 1) * osm_tile_res
        self.z = z
        print (self.w, self.h)
        self.dst_img = mod_pil_image.new ("RGB", (self.w, self.h), 'white')

    def lat_lon_to_image_xy (self, lat_deg, lon_deg):
        """ Internal. Converts lat, lon into dst_img coordinates in pixels """
        lat_rad = mod_math.radians(lat_deg)
        n = 2.0 ** self.z
        xtile_frac = (lon_deg + 180.0) / 360.0 * n
        ytile_frac = (1.0 - mod_math.log(mod_math.tan(lat_rad) + (1 / mod_math.cos(lat_rad))) / mod_math.pi) / 2.0 * n
        img_x = int( (xtile_frac-self.x1)*osm_tile_res )
        img_y = int( (ytile_frac-self.y1)*osm_tile_res )
        return (img_x, img_y)

    def draw_track (self, gpx):
        """ Draw GPX track onto map """
        draw = mod_pil_draw.Draw (self.dst_img)
        trk = 0         # Just changes color of segment a little
        for track in gpx.tracks:
            for segment in track.segments:
                idx = 0
                x_from = 0
                y_from = 0
                for point in segment.points:
                    if (idx == 0):
                        x_from, y_from = self.lat_lon_to_image_xy (point.latitude, point.longitude)
                    else:
                        x_to, y_to = self.lat_lon_to_image_xy (point.latitude, point.longitude)
#                        draw.line ((x_from,y_from,x_to,y_to), (255,0,trk), 2)
                        draw.line ((x_from,y_from,x_to,y_to), 'black', 3)
                        x_from = x_to
                        y_from = y_to
                    idx += 1
                trk += 32
                if (trk > 160):
                    trk = 0
                    
    def save_image(self, filename):
        print("Saving " + filename) 
        self.dst_img.save (filename)


In [276]:
#gpx_files = mod_glob.glob (r"*.gpx")
gpx_files = ['begin.gpx']

In [277]:
for gpx_file in gpx_files:
        try:
            gpx = mod_gpxpy.parse(open(gpx_file))

            # Print some track stats
            print ('--------------------------------------------------------------------------------')
            print ('  GPX file     : %s' % gpx_file)
            start_time, end_time = gpx.get_time_bounds()
            print('  Started       : %s' % start_time)
            print('  Ended         : %s' % end_time)
            print('  Length        : %2.2fkm' % (gpx.length_3d() / 1000.))
            moving_time, stopped_time, moving_distance, stopped_distance, max_speed = gpx.get_moving_data()
            print('  Moving time   : %s' % format_time(moving_time))
            print('  Stopped time  : %s' % format_time(stopped_time))
#            print('  Max speed     : %2.2fm/s = %2.2fkm/h' % (max_speed, max_speed * 60. ** 2 / 1000.))    
            uphill, downhill = gpx.get_uphill_downhill()
            print('  Total uphill  : %4.0fm' % uphill)
            print('  Total downhill: %4.0fm' % downhill)
            min_lat, max_lat, min_lon, max_lon = gpx.get_bounds()
            print("  Bounds        : [%1.4f,%1.4f,%1.4f,%1.4f]" % (min_lat, max_lat, min_lon, max_lon))
            z = osm_get_auto_zoom_level (min_lat, max_lat, min_lon, max_lon, 6)
            print("  Zoom Level    : %d" % z)


            # Create the map
            map_creator = MapCreator (min_lat, max_lat, min_lon, max_lon, z)
            #map_creator.cache_area()
            #map_creator.create_area_background()
            map_creator.draw_track(gpx)
            map_creator.save_image (gpx_file[:-4] + '-' + get_map_suffix() + '.png')
            
        except Exception as e:
            mod_logging.exception(e)
            print('Error processing %s' % gpx_file)
            mod_sys.exit(1) 

--------------------------------------------------------------------------------
  GPX file     : begin.gpx
  Started       : None
  Ended         : None
  Length        : 156.80km
  Moving time   : n/a
  Stopped time  : n/a
  Total uphill  :    0m
  Total downhill:    0m
  Bounds        : [32.7568,32.7980,34.9682,35.0163]
Max tiles: 9
  Zoom Level    : 16
2560 2560
Saving begin-roads.png


In [278]:
pil_image = map_creator.dst_img#.convert('RGB') 
open_cv_image = numpy.array(pil_image) 
# Convert RGB to BGR 
map_orig = cv.cvtColor(numpy.array(pil_image), cv.COLOR_RGB2GRAY)
map_orig.shape


(2560, 2560)

In [291]:
## w, h = template_orig.shape[::-1]
#for scale_percent in [0.1, 0.2, 0.5, 1.0, 5.0, 10.0]:
#for scale_percent in [50, 100, 200, 400, 800]:
for scale_percent in [5, 10, 20]:
    width = int(w * scale_percent / 100)
    height = int(h * scale_percent / 100)
    if(width > 0 and height > 0):
        dim = (width, height)
        template = cv.resize(template_orig, dim, interpolation = cv.INTER_AREA)
        t = template
        t[template == 0] = 1
        t[template > 0] = 0
        mask = t
        # All the 6 methods for comparison in a list
        methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR', 'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
        #methods = ['cv.TM_SQDIFF']
        for meth in methods:
            # img = img_orig.copy()
            img = map_orig.copy()
            method = eval(meth)
            # Apply template Matching
            res = cv.matchTemplate(img,template,method, mask=mask)
            min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
            # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
            if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
                top_left = min_loc
            else:
                top_left = max_loc
            if(max(top_left) > 0):
                bottom_right = (top_left[0] + width, top_left[1] + height)
                cv.rectangle(img,top_left, bottom_right, 128, cv.LINE_4)
                f, axs = plt.subplots(1,2,figsize=(200,200))
                axs[0].imshow(res,cmap = 'gray')
                axs[0].plot(top_left[0], top_left[1], 'ro', markersize=60, linewidth=40)
                #axs[0].title('Matching Result'), plt.xticks([]), plt.yticks([])
                axs[1].set_xlim(0, img.shape[0])
                axs[1].set_ylim(img.shape[1], 0)
                axs[1].imshow(img,cmap = 'gray')
                axs[1].plot(top_left[0], top_left[1], 'ro', markersize=40, linewidth=40)
                extent = [top_left[0], top_left[0]+width, top_left[1]+height, top_left[1]]
                axs[1].imshow(template, cmap='seismic', extent=extent, alpha=0.8)
                #axs[1].title('Detected Point'), plt.xticks([]), plt.yticks([])
                plt.suptitle(meth)
                plt.show()



In [269]:
def cmap_discretize(cmap, N):
    """Return a discrete colormap from the continuous colormap cmap.
    
        cmap: colormap instance, eg. cm.jet. 
        N: number of colors.
    """
    if type(cmap) == str:
        cmap = get_cmap(cmap)
    colors_i = np.concatenate((np.linspace(0, 1., N), (0.,0.,0.,0.)))
    colors_rgba = cmap(colors_i)
    indices = np.linspace(0, 1., N+1)
    cdict = {}
    for ki, key in enumerate(('red','green','blue')):
        cdict[key] = [(indices[i], colors_rgba[i-1,ki], colors_rgba[i,ki]) for i in range(N+1)]
    # Return colormap object.
    return matplotlib.colors.LinearSegmentedColormap(cmap.name + "_%d"%N, cdict, 1024)

In [ ]:
cmap = cmap_discretize(matplotlib.cm.jet, 2)
image = np.random.randn(400, 600)


# Create a sin function from x = 0 - 360, y = -100 - 100
x = np.arange(0, 360, 0.01)
y = 100 * np.sin(np.pi * x / 180)

figure = plt.figure(figsize=(100, 100))
subplot = figure.add_subplot(111)
subplot.set_xlim(0, img.shape[0])
subplot.set_ylim(0, img.shape[1])
subplot.imshow(img, cmap='gray')
subplot.imshow(mask, cmap='seismic', extent=[500, 600, 500, 740], alpha=0.8)

#subplot.plot(x, y)

figure.show()